# Python Tricks 
              Dan Bader 

In [ ]:
# price amount in cents to avoid currency rounding
shoes = {'name': 'Fancy Shoes', 'price': 14900} 

def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price']
    return price

apply_discount(shoes, 0.25), apply_discount(shoes, 0.5), apply_discount(shoes, 1.0), apply_discount(shoes, 1.5)

Assertions are meant to be *internal self-checks*. Pythons's assert statement is a debugging aid, not a mechanism for handling run-time errors.

In [ ]:
# Assertion implementation
if __debug__:
    if not expression1:
        raise AssertionError(expression2)


__debug__ is a global variable which is **true** under normal circumstances and **false** if optimizations are requested.

In computer programming jargon, a **heisenbug** is a software bug that seems to disappear or alter its behavior when one attempts to study it.

Assertions can be globally disabled with -O and -OO command lin switches as wll as `PYTHONOPTIMIZE` environment variable in CPython.

In [ ]:
def delete_product(prod_id, user):
    assert user.is_admin(),
    assert store.has_product(prod_id),
    store.get_product(prod_id).delete()

In [ ]:
# Better version
def delete_product(prod_id, user):
    if not user.is_admin():
        raise AuthError('Must be admin to delete')
    if not store.has_product(prod_id):
        raise ValueError('Unknow product id')
    store.get_product(prod_id).delete()

In [9]:
assert(1 == 2, 'This should fail')

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-9-2c057bd7fe24>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(1 == 2, 'This should fail')


In [10]:
assert 1 == 2, "this should fail"

AssertionError: this should fail

In [11]:
names = [
    'Alice',
    'Bob',
    'Dilbert'
    'Jane'
]
print(names)

['Alice', 'Bob', 'DilbertJane']


In Python, a comma can be placed after every item in a list, dict, or set constant, including the last item.

## Context Manager and with statement

`with` statement simplifies some common resource management patterns by abstracting their functionality and allowing them to be factored out and reused.

Another good example where the `with` statement is used effectively is `threading.Lock` class.

In [ ]:
with open('hello.txt', 'w') as f:
    f.write('Hello World!!')

A class can support `with` statement by implementing so-called _context managers_. Context manager is a simple protocol/interface that your object needs to follow in order to support `with` statement. 

A class/object must add `__enter__` and `__exit__` methods it it want to function as a context manager.


In [ ]:
class ManagedFile:
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

In [ ]:
with ManagedFile('hello.txt') as f:
    f.write('hello world')
    f.write('good byte')

The `contextlib` utility module in the standard library provides a few more abstractions built on top of the basic context manager protocol.

In [12]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):  # generator()
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()
        
with managed_file('hello.txt') as f:
    f.write('hello and byte')

In [14]:
class Indenter:
    def __init__(self):
        self.level = 0
        self.spacing = [
            '....',
            '****',
            '++++',
        ]
    
    def __enter__(self):
        self.level += 1
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1
        
    def print(self, text):
        print('....' * self.level + text)
        

with Indenter() as indent:
    indent.print('hi!')
    with indent:
        indent.print('hello!!')
        with indent:
            indent.print('bonjour')
    indent.print('bye')

....hi!
........hello!!
............bonjour
....bye


**Excercise:** implementing a context manager that measures the execution time of a code block using the time.time function. Be sure to try out writ ing both a decorator-based and a class-based variant to drive home the difference between the two

**Single Leading Underscore:** `_var` (Convention only)
Leading underscores impact how names get imported from modules. Python will _not_ import names with a leading underscore unless the module defines an `__all__` list

**Single Trailing Underscore:** `var_` 
Use by convention to avoid naming conflicts with Python keywords.

**Double Leading Underscore:** `__var` (Name Mangling)
Causes Python interpreter to rewrite the attribute names in order to avoid naming conflicts in subclasses. 

**Double Leading and Trailing Underscore:** `__var__` (No Name Mangling) 
Reserved for special use in the language.

**Single Underscore:** `_`
In REPLs, represents result of last expression evaluated by interpreter.

In [1]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 22

t = Test()
print(t.foo, t._bar)

11 22


In [ ]:
# my_module.py
def external_func():
    return 22

def _internal_func():
    return 42

from my_module import *   # Wild card imports should be avoided
external_func()   # OK
_internal_func()  # NameError

import my_module   # Regular import
my_module.external_func()
my_module._internal_func()  # OK

In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 22
        self.__baz = 44

class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'
        
t = Test()
print(dir(t))

t2 = ExtendedTest()
print(t2.foo, t2._bar, t2.__baz)

Most important skills for a programmer is "pattern recognition" and knowing where to look things up.

Double underscores are often referred to as **"dunders"** in Python community.

In [5]:
car = ('red', 'auto', 12, 3812.4)
color, _, _, mileage = car # _ as placeholder

Strings in Python have a unique built-in operation that can be accessed with the **% operator**.

In [11]:
errno = 50159747054
name = 'Bob'

# old style string formatting
print('Hey %s, there is a 0x%x error!' % (name, errno))
print('Hey %(name)s, there is a 0x%(errno)x error!' % {"name": name, "errno": errno })

# New style string formatting
print('Hey {}, there is a 0x{:x} error!'.format(name, errno))
print('Hey {name}, there is a 0x{errno:x} error!'.format(name=name, errno=errno))

# Literal string interpolation (Python 3.6+)


Hey Bob, there is a 0xbadc0ffee error!
Hey Bob, there is a 0xbadc0ffee error!
Hey Bob, there is a 0xbadc0ffee error!
Hey Bob, there is a 0xbadc0ffee error!


In [12]:
def greet(name, question):
    return f"Hello, {name}! How's it {question}?"

# gets translated into
# def greet(name, question):
#     return ("Hello, " + name + " ! How's it " + question + "?")

import dis
dis.dis(greet)

  2           0 LOAD_CONST               1 ('Hello, ')
              2 LOAD_FAST                0 (name)
              4 FORMAT_VALUE             0
              6 LOAD_CONST               2 ("! How's it ")
              8 LOAD_FAST                1 (question)
             10 FORMAT_VALUE             0
             12 LOAD_CONST               3 ('?')
             14 BUILD_STRING             5
             16 RETURN_VALUE


**grokking**: understand (something) intuitively or by empathy.
"corporate leaders seemed to grok this concept fairly quickly"

In [13]:
def yell(text):
    return text.upper() + '!'

bark = yell
del yell
print(bark('hi'))
print(bark.__name__)

HI!
yell


Python attaches a string identifier to every function at creation
time for debugging purposes. You can access this internal identifier with the `__name__` attribute.

Functions that can accept other functions as arguments are also called _higher-order functions_. They are a necessity for the functional programming style.

Python allows functions to be defined inside other functions.

The inner functions can also _capture and carry some of the parent function's state_ with them. Functions that captures parent's state are called **lexical closures** or just **closures**.

In [19]:
funcs = [bark, str.lower, str.capitalize]
for f in funcs:
    print(f, f('hey there'))
funcs[0]('hello!!')

def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)
    
greet(bark), greet(str.lower)

<function yell at 0x7fb2e85cbaf0> HEY THERE!
<method 'lower' of 'str' objects> hey there
<method 'capitalize' of 'str' objects> Hey there
HI, I AM A PYTHON PROGRAM!
hi, i am a python program


(None, None)

In [20]:
list(map(bark, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

In [21]:
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)
print(speak('Hello, World'))

# whisper is not accessible outside speak
whisper('Yo')   # Error
speak.whisper   # Error

hello, world...


In [24]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper
    
f = get_speak_func(0.3)
print(f, f('hello'))

g = get_speak_func(0.7)
print(g, g('bye'))

<function get_speak_func.<locals>.whisper at 0x7fb2e85cb430> hello...
<function get_speak_func.<locals>.yell at 0x7fb2e85cbe50> BYE!


In [26]:
def get_speak_func(text, volume):
    def whisper():
        # inner function captures the 'text' argument of parent
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper
    
get_speak_func('Hello World', 0.8)()
    

'HELLO WORLD!'

In [27]:
def make_adder(n):
    def add(x):
        return x+n
    return add

plus_3 = make_adder(3)
plus_5 = make_adder(5)

print(plus_3(10))
print(plus_5(10))

13
15


class objects can be made _callable_, which allows objects to be treated like a functions. If an object is callable it means you can use the round parentheses function call syntax on it and even pass in function call arguments.

In [29]:
class Adder:
    def __init__(self, n):
        self.n = n
    
    def __call__(self, x):
        return self.n + x
    
plus_3 = Adder(3)
print(plus_3(4)) # call object instance as function 

callable(plus_3)

7


True

The `lambda` keyword in Python provides a shortcut for declaring small anonymous functions. Lambda functions are restricted to a single expression. Lambda evaluates its expression and then automatically returns the expression's result.

In [30]:
(lambda x, y: x+y)(5, 4) # function expression

9

In [31]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
print(sorted(tuples))
print(sorted(tuples, key=lambda x: x[1]))

[(1, 'd'), (2, 'b'), (3, 'c'), (4, 'a')]
[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]


In [33]:
print(sorted(range(-5, 6)))
print(sorted(range(-5, 6), key=lambda x: x*x))

[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]


Python’s **decorators** allow you to extend and modify the
behavior of a callable (functions, methods, and classes) without permanently modifying the callable itself.

Python's decorators _"decorate”_ or _“wrap”_ another function
and let you execute code _before_ and _after_ the wrapped function runs.

A decorator is a _callable that takes a callable as input and returns another callable_.

Putting an @null_decorator line in front of the function definition is the same as defining the function first and then running through the decorator.

Python’s `*args` and `**kwargs` feature for dealing with variable
numbers of arguments. `*` and `**` collect all positional and keyword arguments and stores them in variables `args` and `kwargs`. 

In [37]:
# Null decorator
def null_decorator(func):
    return func

def greet():
    return 'Hello'

greet = null_decorator(greet)

@null_decorator   # same as above explicit initialization
def greet():
    return 'Hello!'

print(greet())

# Another decorator
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

@uppercase
def greet():
    return 'Hello'

print(greet())

Hello!
HELLO


In [40]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

# Both are same 
# decorated_greet = strong(emphasis(greet))

@strong
@emphasis
def greet():
    return 'Hello!'

print(greet())

<strong><em>Hello!</em></strong>


In [44]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*argc, **kwargs)
    return wrapper


def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}()'
              f' with {args}, {kwargs}')
        
        original_result = func(*args, **kwargs)
        
        print(f'TRACE: {func.__name__}()' 
             f'returned {original_result}')
        return original_result
    return wrapper


@trace
def say(name, line):
    return f'{name}: {line}'

print(say('ABC', 'def'))

TRACE: calling say() with ('ABC', 'def'), {}
TRACE: say()returned ABC: def
ABC: def


One downside of decorator is that it "hides" some of the metadata attached to the original (undecorated) function.

Use `functools.wraps` decorator in user defined decorators to copy over the lost metadata from undecorated function.

In [53]:
def greet():
    """Return a friendly greeting"""
    return 'Hello'

def my_decorator(func):
    def wrapper(*args, **kwargs):
        return func().upper() + "!"
    return wrapper

decorated_greet = my_decorator(greet)

# print(greet.__name__)
# print(greet.__doc__)
print(decorated_greet.__name__)
print(decorated_greet.__doc__)

# import functools

# def uppercase(func):
#     @functools.wraps(func)
#     def wrapper():
#         return func().upper()
#     return wrapper

# @uppercase
# def greet():
#     """Return a friendly greeting"""
#     return 'Hello'

# print(greet.__name__)
# print(greet.__doc__)
    

wrapper
None


`args` will collect extra positional arguments as a tuple because the parameter name has a `*` prefix. Likewise, `kwargs` will collect extra keywords arguments as a dictionary because the parameter name has a `**` prefix.



In [58]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

def bar(x, *args, **kwargs):
    kwargs['name'] = 'Aamir'
    args = args + ('extra', )
    foo(x, *args, **kwargs)
    
foo('hi')
foo('hi', 'hello', 1, 2, 3)
foo('hi', 'hello', 1, 2, 3, k1='value', k2=999)
foo('hi', 'hello', 1, 2, 3, k1='value', k2=999)
bar(1, 2, 3, 4, k1='value, k2=101')

hi
hi
('hello', 1, 2, 3)
hi
('hello', 1, 2, 3)
{'k1': 'value', 'k2': 999}
hi
('hello', 1, 2, 3)
{'k1': 'value', 'k2': 999}
1
(2, 3, 4, 'extra')
{'k1': 'value, k2=101', 'name': 'Aamir'}


_Function Argument Unpacking_ using `*` operator.

Puttin a `*` before an iterable in a function call will _unpack_ it and pass its elements as separate positional arguments to the called function.

`**` operator for unpacking keyword arguments from dictionaries.

In [62]:
def print_vector(x, y, z):
    print('<%s, %s, %s>' % (x, y, z))
    
tuple_vec = (1, 0, 1)
list_vec = [1, 2, 1]
print_vector(*tuple_vec)
print_vector(*list_vec)

dict_vec = {'y': 2, 'z': 3, 'x': 1}
print_vector(**dict_vec)

print_vector(*dict_vec)

<1, 0, 1>
<1, 2, 1>
<1, 2, 3>
<y, z, x>


The `==` operator compares by checking for _equality_. The `is` operator, however, compares _identities_. is check if both objects are pointing to the same object.

In [65]:
a = [1, 2, 3]
b = a
c = list(a)  # create a copy of list object

print(a==b)
print(a is b)

print(a==c)
print(a is c)

True
True
True
False


The default "to string" conversion behavior of a class object is basic and lacks details.

Instead of building your own to-string conversion machinery, you’ll
be better off adding the `__str__` and `__repr__` “dunder” methods to your class.

`__str__` is one of Python’s “dunder” (double-underscore) methods
and gets called when you try to convert an object into a string through the various means that are available:

inspecting an object in a Python interpreter session simply prints the result of the object’s `__repr__` .

We could copy and paste the string returned by `__repr__` and execute
it as valid Python to recreate the original date object.

In [73]:
# class Car:
#     def __init__(self, color, mileage):
#         self.color = color
#         self.mileage = mileage
        
# my_car = Car('red', 37281)
# print(my_car)
# my_car

class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
    
    def __repr__(self):
        return '__repr__ for Car'
    
    def __str__(self):
#         return f'a {self.color} car'
        return '__str__ for Car'

my_car = Car('red', 37281)
print(my_car)
# str(my_car)
# '{}'.format(my_car)
my_car

__str__ for Car


__repr__ for Car

In [74]:
print(str(my_car))
print(repr(my_car))

__str__ for Car
__repr__ for Car


In [75]:
import datetime
today = datetime.date.today()
str(today)

'2021-09-10'

In [76]:
repr(today)

'datetime.date(2021, 9, 10)'

In [79]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
    
    def __repr__(self):
#         return f'Car({self.color!r}, {self.mileage!r}')
        return (f'{self.__class__.__name__}('
               f'{self.color!r}, {self.mileage!r})')

my_car = Car('blue', 12345)
print(my_car)
my_car
        

Car('blue', 12345)


Car('blue', 12345)

In Python 2.x there are two types to represent text: `str` , which is limited to the **ASCII** character set, and `unicode`, which is equivalent to Python 3’s `str`.

In [80]:
# Custom Exception
class NameTooShortError(ValueError):
    pass

def validate(name):
    if len(name) < 10:
        raise NameTooShortError(name)
        
validate('ali')

NameTooShortError: ali

Assignment statements in Python do not create copies of objects, they
only bind names to an object. For immutable objects, that usually
doesn’t make a difference.

In [ ]:
# Copy built-in mutable collections
new_list = list(original_list)
new_dict = dict(original_dict)
new_set = set(original_set)

A **shallow copy** means constructing a new collection object and then
populating it with references to the child objects found in the original. In essence, a shallow copy is only _one level deep_. The copying process does not recurse and therefore won’t create copies of the child objects themselves.

A **deep copy** makes the copying process recursive.

The `copy.copy()` function creates shallow copies of objects.

The `copy.deepcopy()` function creates a deep copy of object.

objects can control how they’re copied by defining the special methods `__copy__()` and `__deepcopy__()` on them.

In [82]:
xs = [
        [1, 2, 3], 
        [4, 5, 6],
        [7, 8, 9]
     ]
ys = list(xs) # shallow copy
print(xs)
print(ys)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [83]:
xs.append(['new sublist'])
print(xs)
print(ys)

[[1, 2, 3], [4, 5, 6], [7, 8, 9], ['new sublist']]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [84]:
xs[1][0] = 'X'
print(xs)
print(ys)

[[1, 2, 3], ['X', 5, 6], [7, 8, 9], ['new sublist']]
[[1, 2, 3], ['X', 5, 6], [7, 8, 9]]


In [85]:
import copy
xs = [
        [1, 2, 3], 
        [4, 5, 6],
        [7, 8, 9]
     ]
zs = copy.deepcopy(xs)
xs[1][0] = 'X'
print(xs)
print(zs)

[[1, 2, 3], ['X', 5, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [87]:
import copy

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point({self.x!r}, {self.y!r})'
    
a = Point(10, 20)
b = copy.copy(a)
print(a)
print(b)
print(a is b)

Point(10, 20)
Point(10, 20)
False


In [90]:
class Rectangle:
    def __init__(self, topleft, bottomright):
        self.topleft = topleft
        self.bottomright = bottomright
        
    def __repr__(self):
        return (f'Rectangle({self.topleft!r},' 
               f'{self.bottomright!r})')
    
rect = Rectangle(Point(0, 1), Point(5, 6))
srect = copy.copy(rect)
print(rect)
print(srect)
print(rect is srect)

rect.topleft.x = 999
print(rect)
print(srect)


Rectangle(Point(0, 1),Point(5, 6))
Rectangle(Point(0, 1),Point(5, 6))
False
Rectangle(Point(999, 1),Point(5, 6))
Rectangle(Point(999, 1),Point(5, 6))


In [92]:
drect = copy.deepcopy(srect)
drect.topleft.x = 222
print(drect)
print(srect)

Rectangle(Point(222, 1),Point(5, 6))
Rectangle(Point(999, 1),Point(5, 6))


**Abstract Base Classes (ABCs)** ensure that derived classes implement
particular methods from the base class.

Python’s `abc` module that was added in Python 2.6.

In [94]:
class Base:
    def foo(self):
        raise NotImplementedError()
    
    def bar(self):
        raise NotImplementedError()
        
class Concrete(Base):
    def foo(self):
        return 'foo() called'

b = Base()
b.foo()
c = Concrete()

NotImplementedError: 

In [96]:
from abc import ABCMeta, abstractmethod

class Base(metaclass=ABCMeta):
    @abstractmethod
    def foo(self):
        pass
    
    @abstractmethod
    def bar(self):
        pass
    
class Concrete(Base):
    def foo(self):
        pass
    
b = Base()


TypeError: Can't instantiate abstract class Base with abstract methods bar, foo

In [97]:
c = Concrete()

TypeError: Can't instantiate abstract class Concrete with abstract methods bar

Namedtuples can be a great alternative to defining a class manually.

Python’s tuples are a simple data structure for grouping arbitrary
objects. Tuples are also immutable. One downside of plain tuples is that the data you store in them can only be pulled out by accessing it through integer indexes.

**namedtuples** are immutable containers, just like regular tuples.

Each object stored in **namedtuples** can be accessed through a unique (human-readable) identifier. namedtuple’s factory function calls split() on the field names string to parse it into a list of field names.

namedtuples are a memory efficient shortcut to defining an immutable class in Python manually.

namedtuple usefull attributes:
_fields
_asdict()
_replace
_make

In [104]:
from collections import namedtuple
# first parameter 'Car' is referred to as "typename"
Car = namedtuple('Car', 'color mileage')
Car = namedtuple('Car', ['color', 'mileage'])
my_car = Car('gray', 73200)
print(my_car.color)
print(my_car.mileage)
print('')
print(my_car[0])
print(my_car[1])

tuple(my_car)
c, m = my_car
print(c, m)

print(*my_car)
my_car

gray
73200

gray
73200
gray 73200
gray 73200


Car(color='gray', mileage=73200)

In [107]:
Car = namedtuple('Car', ['color', 'mileage'])

class MyCarWithMethods(Car):
    def hexcolor(self):
        if self.color == 'red':
            return '#ff0000'
        else:
            return '#000000'
        

c = MyCarWithMethods('gray', 2300)
print(c.hexcolor())

c = MyCarWithMethods('red', 1234)
print(c.hexcolor())

#000000
#ff0000


In [120]:
Car = namedtuple('Car', ['color', 'mileage'])
ElectricCar = namedtuple('ElectricCar', Car._fields + ('charge', ))
e = ElectricCar('red', 1234, 45.0)
print(e)

print(e._asdict())

import json
json.dumps(e._asdict())

e._replace(color='blue')

ElectricCar(color='red', mileage=1234, charge=45.0)
{'color': 'red', 'mileage': 1234, 'charge': 45.0}


ElectricCar(color='blue', mileage=1234, charge=45.0)

**Class variables** (static variables) are declared inside the class definition (but outside of any instance methods).

**Instance variables** are always tied to a particular object instance.

Trying to modify a class variable through an object instance (jack.num_legs = 6), which then accidentally creates an instance variable of the same name, shadowing the original class variable—is a bit of an OOP pitfall in Python.

In [122]:
class Dog:
    num_legs = 4   # <- Class variable
    
    def __init__(self, name):
        self.name = name # <- Instance variable
        
jack = Dog('Jack')
jill = Dog('Jill')
print(jack.name, jill.name)
print(jack.num_legs, jill.num_legs)
print(Dog.num_legs)

Jack Jill
4 4
4


In [124]:
jack.num_legs = 6 # create extra instance variable in Jack instance
print(jack.num_legs, Dog.num_legs)
print(jack.num_legs, jack.__class__.num_legs)

6 4
6 4


In [ ]:
class CountedObject:
    num_instances = 0
    
    def __init__(self):
        self.__class__.num_instances += 1

**Instance methods** can also access the class itself through the `self.__class__` attribute. This means instance methods can also modify class state.

**class methods** take a `cls` parameter that points to the class—and not the object instance — when the method is called.

everything in Python is an object, even classes themselves.

**class methods** allow us to define alternative constructors for our classes. Python only allows one `__init__` method per class. Using class methods makes it possible to add as many alternative constructors as necessary.

One use people have found for class methods is to create inheritable alternative constructors.


@property ?? check docs

In [11]:
class MyClass:
    def method(self):
        return 'instance method called', self
    
    @classmethod
    def classmethod(cls):
        return 'class method called', cls
    
    @staticmethod
    def staticmethod():
        return 'static method called'
    
obj = MyClass()
print(obj.method())

# pass instance object manually
print(MyClass.method(obj))

print(obj.classmethod())

print(obj.staticmethod())

print('')

print(MyClass.classmethod())
print(MyClass.staticmethod())
# print(MyClass.method())

('instance method called', <__main__.MyClass object at 0x7f29e005f8e0>)
('instance method called', <__main__.MyClass object at 0x7f29e005f8e0>)
('class method called', <class '__main__.MyClass'>)
static method called

('class method called', <class '__main__.MyClass'>)
static method called


In [14]:
import math
class Pizza:
    def __init__(self, ingredients, radius=10):
        self.ingredients = ingredients
        self.radius = radius
        
    def __repr__(self):
        return f'Pizza({self.ingredients!r}, {self.radius!r})'
    
    @classmethod
    def margherita(cls):
        return cls(['mozzerella', 'tomatoes'])
    
    @classmethod
    def prosciutto(cls):
        return cls(['mozzarella', 'tomatoes', 'ham'])
    
    def area(self):
        return self.circle_area(self.radius)
    
    @staticmethod
    def circle_area(r):
        return r ** 2 * math.pi
    
print(Pizza(['cheese', 'tomatoes']))
print(Pizza.margherita())
print(Pizza.prosciutto())

Pizza(['cheese', 'tomatoes'], 10)
Pizza(['mozzerella', 'tomatoes'], 10)
Pizza(['mozzarella', 'tomatoes', 'ham'], 10)


In [23]:
# classmethod example
class X:
    def __init__(self):
        pass
    
    def c1(*args):
        print(f'c1 {args}')
        
    c1 = classmethod(c1) # Use classmethod decorator 
    
    @classmethod
    def c2(*args):
        print(f'c2 {args}')
              
inst = X()
X.c1()
X.c2()

inst.c1()
inst.c2()

print('\n\n')

# Alternative constructors example
class Y:
    def __init__(self, astr):
        self.s = astr
        
    @classmethod
    def fromlist(cls, alist):
        x = cls('')
        x.s = ','.join(str(s) for s in alist)
        return x
    
    def __repr__(self):
        return f'Y({self.s!r})'
    
y1 = Y('abc')
print(y1)
y2 = Y.fromlist(range(4))
print(y2)

# subclass Y
class Z(Y):
    def __repr__(self):
        return f'Z({self.s.upper()!r})'

print('\n')
z1 = Z.fromlist(['a', 'b', 'c'])
print(z1)

c1 (<class '__main__.X'>,)
c2 (<class '__main__.X'>,)
c1 (<class '__main__.X'>,)
c2 (<class '__main__.X'>,)



Y('abc')
Y('0,1,2,3')


Z('A,B,C')


Book: **The Algorithm Design Manual** Steven S. Skiena’s

**Dictionaries** are also often called _maps_, _hashmaps_, _lookup tables_, or _associative arrays_. They allow for the efficient lookup, insertion, and deletion of any object associated with a given key.

`collections.OrderedDict` remember the insertion order of keys.

While standard dict instances preserve the insertion order of keys in
CPython 3.6 and above.

The `defaultdict` class is another dictionary subclass that accepts
a callable in its constructor whose return value will be used if a
requested key cannot be found.

The `collections.ChainMap` data structure groups multiple dictionar-
ies into a single mapping.

In [25]:
phonebook = {
    'bob': 1234,
    'alice': 5678,
    'jack': 2468,
}

# dictonary comprehension
squares = {x: str(x*x) for x in range(6)}
print(squares)

{0: '0', 1: '1', 2: '4', 3: '9', 4: '16', 5: '25'}


In [29]:
import collections
d = collections.OrderedDict(one=1, two=2, three=3, four=4)
print(d)
print(d.keys())
print('')

d2 = {'one': 1, 'two':2, 'three':3, 'four':4}
print(d2)
print(d2.keys())

OrderedDict([('one', 1), ('two', 2), ('three', 3), ('four', 4)])
odict_keys(['one', 'two', 'three', 'four'])

{'one': 1, 'two': 2, 'three': 3, 'four': 4}
dict_keys(['one', 'two', 'three', 'four'])


In [33]:
from collections import defaultdict
dd = defaultdict(list)


from collections import ChainMap
d1 = {'one': 1, 'two': 2}
d2 = {'three': 3, 'four': 4}
chain = ChainMap(d1, d2)

print(chain)
print(chain['four'])
print(chain['one'])
# print(chain['missing'])

ChainMap({'one': 1, 'two': 2}, {'three': 3, 'four': 4})
4
1


Python’s lists are implemented as _dynamic arrays_ behind the scenes.

Python’s `array` module provides space-efficient storage of basic C-
style data types like bytes, 32-bit integers, floating point numbers, and so on.

`str` is a recursive data structure—each character in a string is a str object of length 1 itself.

`bytes` objects are immutable sequences of single bytes.
`bytearray` is a _mutable byte array_.

In [37]:
import array
arr = array.array('f', (1.0, 1.5, 2.0, 2.5))
print(arr)
arr[1] = 1.75
print(arr)

b = bytes((0, 1, 2, 3))
print(b)
#b[0] = 1 # Error, immutable

arr = bytearray((0, 1, 2, 3, 4))
print(arr)

array('f', [1.0, 1.5, 2.0, 2.5])
array('f', [1.0, 1.75, 2.0, 2.5])
b'\x00\x01\x02\x03'
bytearray(b'\x00\x01\x02\x03\x04')


`typing.NamedTuple` class added in Python 3.6 is the younger sibling of the namedtuple class in the collections module. It is very simi-
lar to namedtuple , the main difference being an updated syntax for
defining new record types and added support for type hints.

The `struct.Struct` class converts between Python values and C
structs serialized into Python bytes objects. For example, it can be
used to handle binary data stored in files or coming in from network
connections.

In [39]:
from struct import Struct
MyStruct = Struct('i?f')
data = MyStruct.pack(23, False, 42.)
print(data)
MyStruct.unpack(data)

b'\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00(B'


(23, False, 42.0)

A **set** is an unordered collection of objects that does not allow duplicate elements. sets are used to quickly test a value for mem-
bership in the set, to insert or delete new values from a set, and to
compute the union or intersection of two sets.

The **frozenset** class implements an immutable version of set that
cannot be changed after it has been constructed.

In [41]:
vowels = {'a', 'e', 'i', 'o', 'u'}
squares = {x*x for x in range(10)}
print('e' in vowels)
print(vowels)
print(squares)

letters = set('alice')
letters.intersection(vowels)

vowels = forzenset({'a', 'e', 'i', 'o', 'u'})

True
{'e', 'o', 'i', 'u', 'a'}
{0, 1, 64, 4, 36, 9, 16, 49, 81, 25}


{'a', 'e', 'i'}

A short and beautiful algorithm using a stack is depth-first search (DFS) on a tree or graph data structure.

The deque class implements a double-ended queue that supports
adding and removing elements from either end in O(1) time (non-
amortized).

In [46]:
# stack using deque
from collections import deque
s = deque()
s.append('eat')
s.append('sleep')
s.append('code')
s

deque(['eat', 'sleep', 'code'])

In [47]:
s.pop()

'code'

In [48]:
s.pop()

'sleep'

In [49]:
s.pop()

'eat'

In [50]:
s.pop()

IndexError: pop from an empty deque

In [51]:
from queue import LifoQueue
s = LifoQueue()
s.put('eat')
s.put('sleep')
s.put('code')
s

In [52]:
s.get(), s.get(), s.get()

('code', 'sleep', 'eat')

A priority queue is a container data structure that manages a set of
records with totally-ordered 39 keys (for example, a numeric weight
value) to provide quick access to the record with the smallest or largest
key in the set.

**cookie-cutter pattern**?

In [53]:
even_squares = [x * x for x in range(10) if x % 2 == 0]
even_squares

[0, 4, 16, 36, 64]

Python’s iterator protocol: Objects that support the `__iter__` and `__next__` dunder methods automatically work with for-in loops.